In [5]:
'''Test
    1) matrix of polynomials
'''

'Test\n    1) matrix of polynomials\n'

In [6]:
from sage.stats.distributions.discrete_gaussian_integer import DiscreteGaussianDistributionIntegerSampler

import time
import numpy as np
import itertools
from multiprocessing import cpu_count
from datetime import datetime

ncpus=cpu_count()

params = {  512: {'mK': 1024, 'thr': 2.04, 'q': 12289},
            648: {'mK': 1944, 'thr': 2.13, 'q': 3889 },
            768: {'mK': 2304, 'thr': 2.20, 'q': 18433},
            864: {'mK': 2592, 'thr': 2.25, 'q': 10369},
            972: {'mK': 2916, 'thr': 2.30, 'q': 17497},
           1024: {'mK': 2048, 'thr': 2.33, 'q': 12289} }

param = params[512]

mK  = param['mK']
thr0= param['thr']
q   = param['q']

d = euler_phi(mK)
zmstar = [a for a in range(mK) if gcd(a,mK)==1]
K.<z> = CyclotomicField(mK) # WARNING: degree(K) = d
I = identity_matrix(K,2,2)

In [7]:
poly_zero=0*z+0
poly_one=0*z+1

u = 2*z+1
U = matrix([[1,0],[u,1]])

#z1=K.random_element()
#z2=K.random_element()
Z = matrix([[z],[0]])
Z,U,U*Z

(
[z]  [      1       0]  [        z]
[0], [2*z + 1       1], [2*z^2 + z]
)

In [8]:
U.dimensions()

(2, 2)

In [9]:
U[0,0]+1, Z[0,0]+z

(2, 2*z)

# Notation
For $a\in \mathbb{R}$, note  $[a]_p=\frac{\lfloor aq \rceil}{q}\in (1/q)\mathbb{Z}$
# Integral Matrix Gram Root (`IntGram`)
**Input**: Matrix $M\in\mathcal{R}^{2\times 2}$.

**Output**: Matrix $A\in \mathcal{R}^{2\times m}$ such that $M=AA^t$.

# Offline Sampler (`OffSampler`)
Precompute integer $r>\eta_\epsilon (\mathcal{R}^2)$ such that $Lr\geq \eta_\epsilon(\Lambda(A)^{\perp})$.
$A$ is $2\times m$ matrix.

**Input**: integer $p>0$, matrix $A\in \mathcal{R}^{2\times m}$ ($A$ outputs from `IntGram`).

**Output**: $t \in \mathcal{R}^2$ follows $D_{\mathcal{R}^2,r^2\Sigma}$ with $\Sigma = \frac{1}{p^2}AA^t+I$
#  Integer arithmetic ring Gaussian sampler (`IntGaussSampler`)
Choose $r,s$. Precompute $\hat{B}$ such that $\hat{B}U_{\hat{u}}=B=\tilde{B}U_{u}$ with $\hat{u}=[u]_p \in \frac{1}{p}\mathcal{R}$.

**Input**: Matrix $\hat{B}\in \mathcal{R}^{2\times 2}$, vector $c\in \mathcal{R}^2$ and $r,s$.

**Output**: $z\in \mathcal{R}^2$ follows $D_{\mathcal{L}(\mathcal{R}^2),c,rs}$.


In [10]:
def RSamplerZ(c1, r, verbose=False):
    #print(c1)
    center = c1.list()
    l=[]
    for c in center:
        D = DiscreteGaussianDistributionIntegerSampler(c = c, sigma = r)
        l.append(D())
    if verbose:
        print('---[ RingSamplerZ ]---')
        print('Center vector:')
        print(center)
        print('Sampler:')
        print(l)
    return K(l)

def HybridU(c1,c2, u, r, verbose=False):
    '''c1,c2,z2,u : polynomial
    '''
    U = matrix([[1,u],[0,1]])
    z2 = RSamplerZ(c2, r)
    temp = c1-z2*u
    z1 = RSamplerZ(temp, r)
    res = U*matrix([[z1],[z2]])
    
    if verbose:
        print('---[ HybridU ]---')
        print('Matrix U : ')
        print(U)
        print('Vector c : ')
        print(matrix([c1.list(),c2.list()]))
        print('Sample :')
        print(res[0,0].list())
        print(res[1,0].list())
        
    return res

def IntGram(M):
    '''Find A st : AAt = M
       A : 2 x m (general case: nx9n)
       use Cholesky decomposition
       ? coeffs are rational or real, if real : problem for round_p => sol : define KxR (cyclotomic field R)
    '''
    
    '''? find A'''
    
    return A

def GS(B):
    
    #return B_GS, U 
    return U

def precompute(B, p, r, s, verbose = False):
    '''? p is important for existence of A as integral gram root of M
       ? L needs to be chosen 
    '''
    #B_GS, U = GS(B)
    
    if verbose:
        return random_matrix(K,2,2), K.random_element(), random_matrix(K,2, m=10), 2*r
    else:
        U = GS(B)
        u_hat = round_p(f = U[0,1], p = p)

        B_hat = matrix([[B[0,0], B[0,1]-B[0,0]*u_hat], [B[1,0], B[1,1]-B[1,0]*u_hat]])

        Sigma_p = s*s*I - B_hat*(B_hat.transpose())
        M = p*p*(Sigma_p - I)

        A = IntGram(M)
        L = 2*r
        return B_hat, u_hat, A, L

def OffSampler(p, A, r, L):
    '''r > smoothing(R^2)
       Lr > smoothing(...)
       A : 2 x m
    '''
    
    #Sigma = RDL(1/(p*p)*A*A.transpose()+I)
    '''? need RDL: I prefer staying in Q if possible
       ? RDL compatible with matrix
    '''
    
    m = A.ncols()
    delta = L*r
    D = DiscreteGaussianDistributionIntegerSampler(sigma = delta)
    # x : m x 1
    x = matrix([D() for _ in range(m)]).transpose()
    
    # tt : 2 x 1
    tt = K(1/(p*L))*A*x
    #print(tt)
    t = HybridU(c1 = tt[0,0], c2 = tt[1,0], u = 0, r = r, verbose = False)
    
    return t
    
    
def IntGaussSampler(c, p, B_hat, u_hat, A, r, s ):
    '''? B_hat needs to be inversible in R^{2x2} if not use inverse of B_hat in K^{2x2}
    '''
    #B_hat, u_hat, A, L = precompute(B = B, p = p, r = r, s = s)
    t = OffSampler(p = p, A = A, r = r, L = L)
    
    c_hat = (B_hat.inverse_of_unit())*(c-t)
    zz = HybridU(c1 = c_hat[0,0], c2 = c_hat[1,0], u = u_hat, r = r, verbose=False)
    
    return B_hat*zz

def round_p(f, p):
    '''f must be in K (coeffs are rational)
       
    '''
    coeffs = f.list()
    return K( [round(coeff*p)/p for coeff in coeffs])


    


In [30]:
'''Test RingSamplerZ
'''
K.<z> = CyclotomicField(7)
RSamplerZ(c1=3*z^2+1,r=0.5, verbose=True)

---[ RingSamplerZ ]---
Center vector:
[1, 0, 3, 0, 0, 0]
Sampler:
[1, 0, 4, 0, 0, 1]


z^5 + 4*z^2 + 1

In [12]:
'''Test HybridU
'''
K.<z> = CyclotomicField(7)
HybridU(c1=z^3-2*z+1,c2=-z^2-5, u=z^4-1, r=0.4, verbose=True)

---[ HybridU ]---
Matrix U : 
[      1 z^4 - 1]
[      0       1]
Vector c : 
[ 1 -2  0  1  0  0]
[-5  0 -1  0  0  0]
Sample :
[1, -2, 0, 1, 0, 0]
[-5, 0, -1, -1, 0, 0]


[ z^3 - 2*z + 1]
[-z^3 - z^2 - 5]

In [13]:
'''Test IntGaussSampler
   not yet: since IntGram not implemented => no A
'''
K.<z> = CyclotomicField(7)
c = random_matrix(K,2,1)
B = random_matrix(K,2,2)
p = 10
r = 0.5
s = 1

B_hat, u_hat, A, L = precompute(B=B, p=p, r=r, s=s, verbose=True)
v=IntGaussSampler(c, p, B_hat, u_hat, A, r, s)

In [14]:
c.list()

[-1/2*z^4 - z^3 + z^2 + z - 1/2, -z^4 - z^3 - 2*z^2 - z]

In [15]:
v.list()

[5/4*z^5 - 7/4*z^4 - 3/2*z^3 + 15/4*z^2 + 3/2*z - 1,
 -3*z^5 - 7/2*z^4 - 7/4*z^3 - 21/4*z^2 + 1/2*z - 9/4]

In [21]:
t=vector([i for i in range(2)])
t.norm()

1

In [24]:
t.transpose()

AttributeError: 'sage.modules.vector_integer_dense.Vector_integer_dense' object has no attribute 'transpose'

In [27]:
m = Matrix([[1, 1, 0]])
m.transpose()

[1]
[1]
[0]

In [ ]:
def RabinShallit(N):
    x = [x1,x2,x3,x4]
    return x

def gadget(g, a)

def DiagDomIGD(M,d,b,k):            #d,b,k need conditions, version over Z, not yet over ring # M : nxn
    g = vector([b**i for i in range(k)])
    x = RabinShallit(d - M[0,0] - g.norm()**2)      #norm needs reconsider
    
    if n == 1:
        return g,x
    
    C = [gadget(g, M[0,j]) for j in range(1,n)]
    
    

In [265]:
B = random_matrix(K,2,2)
B

[          z^5 - 2*z^3 - z^2 + z - 2      -z^5 + 2*z^4 + z^3 - 2*z^2 + 1]
[2*z^5 - 2*z^4 - 1/2*z^3 + z^2 - 1/2    -2*z^5 - z^4 + z^3 - z^2 + z - 1]

In [309]:
r = matrix(RDF, 5, 5, [ 0,0,0,0,1, 1,1,1,1,1, 16,8,4,2,1, 81,27,9,3,1, 256,64,16,4,1 ])
m = r * r.transpose(); m


[    1.0     1.0     1.0     1.0     1.0]
[    1.0     5.0    31.0   121.0   341.0]
[    1.0    31.0   341.0  1555.0  4681.0]
[    1.0   121.0  1555.0  7381.0 22621.0]
[    1.0   341.0  4681.0 22621.0 69905.0]

In [311]:
 L = m.cholesky(); L

[               1.0                0.0                0.0                0.0                0.0]
[               1.0                2.0                0.0                0.0                0.0]
[               1.0               15.0 10.723805294763608                0.0                0.0]
[               1.0               60.0  60.98581445891652  7.792973423707309                0.0]
[               1.0              170.0 198.62352415518683  39.36656677955255 1.7230995806796983]

In [315]:
L.dimensions()

(5, 5)

this Cholesky decomposition is real

In [312]:
L.parent()

Full MatrixSpace of 5 by 5 dense matrices over Real Double Field

In [313]:
L*L.transpose()

[              1.0               1.0               1.0               1.0               1.0]
[              1.0               5.0              31.0             121.0             341.0]
[              1.0              31.0             341.0            1555.0            4681.0]
[              1.0             121.0            1555.0            7381.0           22621.0]
[              1.0             341.0            4681.0           22621.0 69904.99999999999]

In [314]:
( L*L.transpose() - m ).norm(1) 

1.4551915228366852e-11

In [222]:
K.<z> = CyclotomicField(7)
A = random_matrix(K,2,2)
P,L,U = A.LU(pivot='nonzero')
A

[     z^5 - 2*z^4 - 2*z^3 - z^2 + z - 1/2 -2*z^5 + 2*z^4 - z^3 - 2*z^2 - 1/2*z - 1]
[     2*z^5 - 2*z^4 - 1/2*z^2 + 2*z - 1/2               -1/2*z^3 - 1/2*z^2 + z - 1]

In [223]:
1/2

1/2

In [208]:
P, P.ncols()

(
[1 0]   
[0 1], 2
)

In [200]:
L

[                                                                                       1                                                                                        0]
[-3160/8387*z^5 - 4656/8387*z^4 + 3044/8387*z^3 + 5840/8387*z^2 + 4276/8387*z - 4448/8387                                                                                        1]

In [205]:
U*(U.transpose())

[                                                                                                       -5*z^5 - 5*z^4 - 2*z^3 - 4*z^2 - 21/2*z - 4                                                     48861/8387*z^5 + 85242/8387*z^4 + 69427/8387*z^3 + 105301/16774*z^2 + 1478/8387*z + 41747/8387]
[                                                    48861/8387*z^5 + 85242/8387*z^4 + 69427/8387*z^3 + 105301/16774*z^2 + 1478/8387*z + 41747/8387 -292336564/70341769*z^5 + 1669013024/70341769*z^4 + 1270476665/70341769*z^3 + 2536674232/70341769*z^2 + 1504534224/70341769*z + 583785372/70341769]

In [221]:
K.<z> = CyclotomicField(7)
d = euler_phi(7)
D0 = DiscreteGaussianDistributionIntegerSampler(sigma=2)
m = 1

verbose = True

if verbose:
    print("Generating %d Gaussian samples" % (m))
fs = [ K( [ D0() for _ in range(d) ] ) for _ in range(m) ]
x = matrix([ D0() for _ in range(d) ]).transpose()

A = random_matrix(K,2,d)
A,A.dimensions(),x.dimensions(), A*x

Generating 1 Gaussian samples


(
[                         -2*z^5 + z^3 - 1                                -z^3 + z^2                -1/2*z^5 + 2*z^4 - z^3 - 1                             z^4 + 2*z - 1           1/2*z^5 - z^4 + z^3 + 2*z^2 - 2 1/2*z^5 - z^4 + z^3 - 1/2*z^2 - 1/2*z + 1]
[                      2*z^5 + z^3 - 2*z^2                 1/2*z^3 - z^2 - 2*z - 1/2             z^5 - z^3 - z^2 - 1/2*z - 1/2             1/2*z^5 + z^4 + z^3 + 2*z - 2    -z^5 + 2*z^4 - 2*z^3 + z^2 + 2*z + 1/2                             z^2 + 2*z - 1],

                [    -1/2*z^5 + 5*z^4 + z^3 - 3*z^2 + z - 2]
(2, 6), (6, 1), [7/2*z^5 + z^4 - 3*z^3 + z^2 + 17/2*z - 9/2]
)

In [206]:
I=identity_matrix(K,2,2)
identity_matrix(K,2,2).transpose()
I*(I.transpose())

[1 0]
[0 1]

In [71]:
D0 = DiscreteGaussianDistributionIntegerSampler(c=1,sigma=2)
[ D0() for _ in range(d) ]

[-1,
 4,
 -5,
 2,
 3,
 5,
 1,
 -1,
 0,
 0,
 2,
 -2,
 0,
 3,
 5,
 3,
 -1,
 3,
 3,
 -1,
 -1,
 3,
 3,
 -2,
 3,
 0,
 3,
 -1,
 -1,
 2,
 1,
 4,
 1,
 4,
 -1,
 0,
 1,
 4,
 1,
 3,
 2,
 2,
 2,
 2,
 1,
 1,
 1,
 1,
 0,
 2,
 -2,
 3,
 2,
 3,
 -1,
 0,
 4,
 5,
 3,
 0,
 1,
 -3,
 1,
 4,
 -1,
 1,
 2,
 -2,
 0,
 0,
 -1,
 2,
 0,
 5,
 1,
 1,
 -2,
 0,
 1,
 0,
 3,
 0,
 0,
 -1,
 2,
 2,
 -1,
 -2,
 1,
 0,
 1,
 -1,
 -3,
 0,
 -1,
 2,
 -1,
 -1,
 -2,
 4,
 -1,
 3,
 -1,
 3,
 4,
 3,
 2,
 -2,
 0,
 2,
 2,
 1,
 2,
 3,
 0,
 5,
 1,
 -1,
 5,
 1,
 0,
 1,
 1,
 2,
 -1,
 2,
 4,
 -1,
 2,
 2,
 4,
 2,
 -2,
 2,
 2,
 -1,
 4,
 2,
 -1,
 4,
 0,
 0,
 4,
 1,
 4,
 -1,
 4,
 1,
 3,
 1,
 1,
 2,
 4,
 -3,
 3,
 -3,
 3,
 3,
 -2,
 3,
 -2,
 2,
 1,
 2,
 2,
 2,
 0,
 0,
 1,
 -2,
 0,
 -1,
 2,
 3,
 4,
 0,
 3,
 -2,
 -1,
 0,
 2,
 1,
 -2,
 -2,
 1,
 1,
 0,
 0,
 3,
 1,
 1,
 3,
 -3,
 -1,
 1,
 5,
 3,
 1,
 0,
 2,
 2,
 1,
 0,
 3,
 1,
 -5,
 3,
 2,
 -1,
 1,
 0,
 3,
 1,
 0,
 3,
 1,
 -3,
 3,
 1,
 1,
 4,
 1,
 -1,
 2,
 1,
 3,
 -1,
 5,
 -1,
 -3,
 0,
 3,
 -2,
 5,
 2,
 -2

In [68]:
fs[0].list()[1]

-2

In [85]:
x=3
print("%d \n" %x)

3 



In [233]:
A = MatrixSpace(IntegerRing(),2)([1,10,3,-1])
A.is_invertible()
A

[ 1 10]
[ 3 -1]

In [234]:
~A

[ 1/31 10/31]
[ 3/31 -1/31]

In [235]:
A.inverse_of_unit()

ArithmeticError: non-invertible matrix

In [236]:
A.is_invertible()

False

In [238]:
type(round(0.64))

<class 'sage.rings.integer.Integer'>

In [254]:
K.<z> = CyclotomicField(7)
def round_p(f, p):
    coeffs = f.list()
    return K( [round(coeff*p)/p for coeff in coeffs])

In [259]:
f = K.random_element()
f.list()

[-6, 1/2, 1/2, 1/2, 1/3, 8]

In [262]:
round_p(f, p=9).list()

[-6, 5/9, 5/9, 5/9, 1/3, 8]